# Part I. One-sided finite differences

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a one-sided finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$. 

In [1]:
import numpy as np

In [31]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the one-sided rule of the approximation order of $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """
    
    Y = f(x+h)
    y = f(x)
    
    d = (Y-y)/(h)
    
    return (d)
    

#### Test I.1

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Comment on whether your results are consistent with the expected value of $f'(x) = 0$ and on an expected scaling with $h\to 0$.

 (10% of the total grade)

In [32]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print(h, err)
    

0.01 0.00010000000000000002
0.001 1e-06
0.0001 1.0000000000000002e-08
1e-05 1.0000000000000002e-10


 Мы ожидали, что производная нашей функции в 0 будет равна 0, а также, что погрешность будет тем меньше, чем меньше h, причём, пропорционально h^2. Собственно это мы и получили.
 
 На всякий случай я проверила для x=1 и x=2. Для этих значений я получила такой же результат.

### Test I.2

Now use a slightly more complicated function, $f(x) = x^2 \log{x}$, evaluate the derivative at $x=1$ using your one-sided rule and a two-point one-sided rule. Roughly estimate the value of $h$ where the error stops decreasing, for these two schemes. 
(15% of the total grade)

In [40]:
def der2(f, x, h):
    
    Y = f(x+h)
    y = f(x-h)
    
    d = (Y-y)/(2*h)
    
    return (d)
    

In [41]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

In [45]:
x = 1
for h in [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15, 1e-16]:
    err = deriv(f, x, h)
    err2 = der2(f, x, h)
    fd = fder(x)
    print(h," ", err," ", err2," ", fd)

0.01   1.015033250331677   1.0000333336666771   1.0
0.001   1.0015003332499228   1.0000003333333118   1.0
0.0001   1.0001500033331399   1.0000000033332233   1.0
1e-05   1.0000150000398844   1.0000000000343334   1.0
1e-06   1.0000014999180666   0.9999999999735779   1.0
1e-06   1.0000014999180666   0.9999999999735779   1.0
1e-07   1.0000001505838707   1.0000000000287592   1.0
1e-08   1.0000000089225287   0.9999999994736439   1.0
1e-09   1.0000000842403711   1.00000002722922   1.0
1e-10   1.000000082890371   1.000000082740371   1.0
1e-11   1.000000082755371   1.000000082740371   1.0
1e-12   1.0000889005838414   1.00003338943111   1.0
1e-13   0.9992007221627905   0.9997558336749531   1.0
1e-14   0.9992007221626559   0.9992007221626409   1.0
1e-15   1.1102230246251583   1.054711873393899   1.0
1e-16   0.0   0.5551115123125782   1.0


Замечаем, что для первого метода рост начинается с 10^(-9), а для второго с 10^(-8). При этом второй метод точнее.

### Test I.3 

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 
(25% of the total grade)

In [46]:
def der3(f, x, h):
    
    Y = f(x+2*h)
    y = f(x+h)
    z = f(x)
    
    d = (-3*x/2 + 2*y - Y/2)/h
    
    return d

In [47]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(f, x, h) - fder(x)
    print(h, err)

0.01 -0.04605170185988091
0.001 -0.006907755278982136
0.0001 -0.0009210340371976182
1e-05 -0.0001151292546497023


Это вообще законно, что мы так быстро скатываемся в возрастание ошибки???

# Part II. Midpoint rule 

Write a function which computes a definite integral using the midpoint rule up to a given error, $\epsilon$. Estimate the error by comparing the estimates of the integral at $N$ and $2N$ elementary intervals. 

In [1]:
def midpoint_rule(func, a, b, eps):
    """ Calculate the integral of f from a to b using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    eps : float
        The target accuracy of the estimate.
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    # ... ENTER YOUR CODE HERE ...
    
    # делим на отрезки
    
    f = func
    N = 1
    I = 0
    I2 = 0
    I0 = 1000000000
    delta = 100000000
    
    while delta>= eps:
        for i in range (N):
            l = (b-a)/N
            
            miniI = l*f(a + l/2 + i*l)
            I = I+miniI  
        N = N*2
        delta = abs(I0-I)
        I0 = I
        
    for i in range (2*N):
            l = (b-a)/(2*N)
            miniI2 = l*f(a + l/2 + i*l)
            I2 = I2+miniI2
            
    return I, I2, N

### Test II.1

Test your midpoint rule on a simple integral, which you can calculate by paper and pencil.

Compare the rate of convergence to the expected $O(N^{-2})$ scaling by studying the number of intervals required for a given accuracy $\epsilon$.

Compare the numerical results to the value you calculated by hand. Does the deviation agree with your estimate of the numerical error?
(20% of the total grade)


In [ ]:
def f(x):
    return x+1
I, I2, N = midpoint_rule(f, 0, 1, 10**(-4))
print (I, I2, N)

### Test II.2

Now use your midpoint rule to compute the value of

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

up to a predefined accuracy of $\epsilon=10^{-4}$.

Note that the integral contains an integrable singularity at the lower limit. Do calculations two ways: first, do a straightforward computation; next, subtract the singularity. Compare the number of iterations required to achieve the accuracy of $\epsilon$.

(30% of the total grade)

In [ ]:
from math import sin

def s(x):
    s = sin(x**0.5)/x
    return s

I, I2, N = midpoint_rule(s, 0, 1, 10**(-4))
print(I, I2, N)

Вот где-то тут ошибка, но я никак не могу найти где. Почему-то оно очень странно считает. Не прогоняя все N. Он очень долго считает, я не понимаю, в чём дело. Я умываю руки.